In [ ]:
#  Imports
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

#  Chargement des données
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

#  Extraction du titre depuis le nom
train["Title"] = train["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)
test["Title"] = test["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)

#  Regroupement des titres rares
for dataset in [train, test]:
    dataset["Title"] = dataset["Title"].replace(
        ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev',
         'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset["Title"] = dataset["Title"].replace('Mlle', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Ms', 'Miss')
    dataset["Title"] = dataset["Title"].replace('Mme', 'Mrs')




In [ ]:

#  Remplissage des valeurs manquantes
train["Age"] = train.groupby("Title")["Age"].transform(lambda x: x.fillna(x.median()))
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
test["Age"] = test["Age"].fillna(train["Age"].median())
test["Fare"] = test["Fare"].fillna(train["Fare"].median())

#  Encodage des colonnes catégorielles
le = LabelEncoder()
for col in ["Sex", "Embarked", "Title"]:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

#  Sélection des features
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Title"]
X = train[features]
y = train["Survived"]
X_test = test[features]

#  Séparation en jeu d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle RandomForest avec meilleurs paramètres
model = RandomForestClassifier(

)
model.fit(X_train, y_train)

#  Évaluation sur validation
y_pred = model.predict(X_val)
print(" Accuracy:", accuracy_score(y_val, y_pred))
print("\n Rapport de classification :\n")
print(classification_report(y_val, y_pred))

#  Prédictions finales sur test.csv
predictions = model.predict(X_test)

#  Création du fichier de soumission
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions
})
submission.to_csv("submission5.csv", index=False)
print(" Fichier de soumission créé avec succès !")

#  Répartition des prédictions
prop = submission["Survived"].value_counts(normalize=True) * 100
print("\n Répartition des prédictions (en %) :")
print(prop)

 Accuracy: 0.8379888268156425

 Rapport de classification :

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       105
           1       0.81      0.80      0.80        74

    accuracy                           0.84       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179

 Fichier de soumission créé avec succès !

 Répartition des prédictions (en %) :
Survived
0    61.004785
1    38.995215
Name: proportion, dtype: float64
